<a href="https://colab.research.google.com/github/TheZxSmallboy/CapstoneNav-MavSDK/blob/main/1004448_AI_HW_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch - homework 1


Please run the whole notebook with your code and submit the `.ipynb` file that includes your answers. 

In [1]:
from termcolor import colored

student_number="1004448"
student_name="Wang Zixuan"

print(colored("Homework by "  + student_name + ', number: ' + student_number,'red'))

Homework by Wang Zixuan, number: 1004448


 ## Question 1 -- matrix multiplication

Implement the following mathematical operation on both the CPU and GPU (use Google Colab or another cloud service if you don't have a GPU in your computer). Print:

a) which type of GPU card you have and 

b) show the computation time for both CPU and GPU (using PyTorch). 

c) How much % faster is the GPU? 

 The operation to implement is the dot product $C = B * A^T$

 whereby $A$ is a random matrix of size $20,000 \times 1,000$ and $B$ is a random matrix of size $2,000 \times 1,000$. In addition to the required information asked above:
 
 d) please also print the resulting two $C$ matrices (they should be the same btw). 
 



In [10]:
# implement solution here
import torch

# a) The type of GPU card you have
gpu_card = torch.cuda.get_device_name(0)
print(torch.cuda.is_available())
print("a) The GPU card is", gpu_card)

A = torch.rand((20000, 1000))
B = torch.rand((2000, 1000))

start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

# b) and d) GPU computational time and C matrix
device = 'cuda'
start.record()
C = torch.matmul(B.to(device), torch.transpose(A,0,1).to(device))
end.record()
torch.cuda.synchronize()
time_elapsed1 = start.elapsed_time(end)
print("GPU compuation time is", time_elapsed1)
print(C)

# b) and d) CPU computational time and C matrix
device = 'cpu'
start.record()
C = torch.matmul(B.to(device), torch.transpose(A,0,1).to(device))
end.record()
torch.cuda.synchronize()
time_elapsed2 = start.elapsed_time(end)
print("CPU compuation time is", time_elapsed2)
print(C)

# c) Percentage faster for GPU
time_diff = ((time_elapsed2- time_elapsed1)/time_elapsed2)*100

print("GPU is", time_diff, "% faster")





True
a) The GPU card is Tesla T4
GPU compuation time is 48.00947189331055
tensor([[261.8482, 256.2710, 251.7561,  ..., 254.4596, 251.9306, 259.3327],
        [252.9153, 247.4323, 244.0038,  ..., 243.6679, 244.7730, 251.9611],
        [247.6445, 242.4991, 243.2641,  ..., 243.0279, 238.0548, 250.7950],
        ...,
        [245.5389, 238.2775, 238.5473,  ..., 240.8333, 240.2434, 251.8797],
        [256.1284, 244.4170, 246.7598,  ..., 251.2768, 245.2113, 253.2175],
        [244.3440, 237.8811, 240.4344,  ..., 236.6896, 237.1082, 249.4396]],
       device='cuda:0')
CPU compuation time is 1014.7877807617188
tensor([[261.8484, 256.2712, 251.7562,  ..., 254.4597, 251.9305, 259.3327],
        [252.9152, 247.4322, 244.0037,  ..., 243.6679, 244.7731, 251.9609],
        [247.6446, 242.4991, 243.2640,  ..., 243.0279, 238.0548, 250.7951],
        ...,
        [245.5389, 238.2777, 238.5475,  ..., 240.8333, 240.2434, 251.8795],
        [256.1286, 244.4169, 246.7599,  ..., 251.2767, 245.2115, 253.2178

## Question 2 - grad


Find the gradient (partial derivatives) of the function $g(w)$ below. 

Let  $w=[w_1,w_2]^T$

Consider  $g(w)=2w_1w_2+w_2cos(w_1)$

a) In PyTorch, compute:   $\nabla g(w)$ 

 and verify that $\nabla g([\pi,1])=[2,2\pi−1]^T$ using the grad function, whereby the first position is the partial for $w_1$ and the second position is the partial for $w_2$. 

b) You can also write a function to manually calculate these partial derivatives! You can review your differential equations math at [here](https://www.wolframalpha.com/input/?i=derivative+y+cos%28x%29) and implement this as a second function below to verify that it comes to the same solution. 


In [17]:
# write your solution here
import math

def g(w):
  return 2*w[0]*w[1] + w[1]*torch.cos(w[0])

## Part A
W = [math.pi, 1]
w = torch.tensor(W, requires_grad=True)
grad = torch.autograd.grad(g(w), w)[0]
print("Part A Pytorch calculated:", grad)

## Part B manually calculate
grad_manual = torch.Tensor([2,2*math.pi -1])
print("Part B Manually calculated:", grad_manual)


Part A Pytorch calculated: tensor([2.0000, 5.2832])
Part B Manually calculated: tensor([2.0000, 5.2832])


## Question 3 - dance hit song prediction

Implement logistic regression in PyTorch for the following dance hit song prediction training dataset: 
https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030training.csv

 * Input variables: a number of audio features (most already standardized so don't worry about that)
 * Target variable: Topclass1030: 
   * 1 means it was a top 10 hit song; 
   * 0 means it never went above top 30 position.

This dataset is derived from my paper on dance hit song prediction, for full description of features have a look at https://arxiv.org/abs/1905.08076. 

Print the evolution of the loss every few epochs and train the model until it converges. 
 
 After training the logistic regression model, calculate the prediction accuracy on the test set: 
 https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030test.csv








In [36]:
# Get the dataset
# !wget https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030training.csv
# !wget https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030test.csv

# Your code here
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F

# load data
training_data = pd.read_csv("/content/herremans_hit_1030training.csv")
testing_data = pd.read_csv("/content/herremans_hit_1030test.csv")

# define logistic regression model
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_size, num_classes)
    def forward(self, x):
        out = self.linear(x)
        out = torch.sigmoid(out)
        return out

# train model
device = "cuda"
epochs = 10001

output_size = 1
input_size = 49 

logreg_clf = LogisticRegression(input_size, output_size).to(device)
learning_rate = 0.001
loss_function = torch.nn.BCELoss()
optimizer = torch.optim.SGD(logreg_clf.parameters(), lr= learning_rate)

for epoch in range(epochs):
    x = torch.FloatTensor(training_data.loc[:, training_data.columns != 'Topclass1030'].values).to(device)
    y = torch.FloatTensor(training_data['Topclass1030']).to(device)
    optimizer.zero_grad()
    y_pred = logreg_clf(x)    
    loss = loss_function(y_pred, y.view(-1,1)).to(device)
    loss.backward()
    optimizer.step()
    if epoch % 500 == 0:
        print ("Epoch:", epoch, "Loss:{0}".format(loss))


Epoch: 0 Loss:0.6603489518165588
Epoch: 500 Loss:0.6312949657440186
Epoch: 1000 Loss:0.6134909391403198
Epoch: 1500 Loss:0.599833607673645
Epoch: 2000 Loss:0.588792085647583
Epoch: 2500 Loss:0.5796651244163513
Epoch: 3000 Loss:0.571998655796051
Epoch: 3500 Loss:0.5654703378677368
Epoch: 4000 Loss:0.559844970703125
Epoch: 4500 Loss:0.5549471974372864
Epoch: 5000 Loss:0.5506445169448853
Epoch: 5500 Loss:0.5468347072601318
Epoch: 6000 Loss:0.5434380173683167
Epoch: 6500 Loss:0.540391206741333
Epoch: 7000 Loss:0.5376434922218323
Epoch: 7500 Loss:0.5351535081863403
Epoch: 8000 Loss:0.5328873991966248
Epoch: 8500 Loss:0.5308170318603516
Epoch: 9000 Loss:0.5289186835289001
Epoch: 9500 Loss:0.5271724462509155
Epoch: 10000 Loss:0.5255614519119263


Run the below code to test the accuracy of your model on the training set: 

In [37]:
import pandas as pd 

test = pd.read_csv('/content/herremans_hit_1030test.csv')
labels = test.iloc[:,-1]
test = test.drop('Topclass1030', axis=1)
testdata = torch.Tensor(test.values)
testlabels = torch.Tensor(labels.values).view(-1,1)
device="cuda"

TP = 0
TN = 0
FN = 0
FP = 0

for i in range(0, testdata.size()[0]): 
  # print(testdata[i].size())
  Xtest = torch.Tensor(testdata[i]).to(device)
  y_hat = logreg_clf(Xtest)
  
  if y_hat > 0.5:
    prediction = 1
  else: 
    prediction = 0

  if (prediction == testlabels[i]):
    if (prediction == 1):
      TP += 1
    else: 
      TN += 1

  else:
    if (prediction == 1):
      FP += 1
    else: 
      FN += 1

print("True Positives: {0}, True Negatives: {1}".format(TP, TN))
print("False Positives: {0}, False Negatives: {1}".format(FP, FN))
rate = TP/(FN+TP)
print("Class specific accuracy of correctly predicting a hit song is {0}".format(rate))

True Positives: 44, True Negatives: 14
False Positives: 15, False Negatives: 6
Class specific accuracy of correctly predicting a hit song is 0.88
